In [79]:
!pip install python-dotenv
!pip install azure-search-documents

from dotenv import load_dotenv
from openai import AzureOpenAI
import os

# Load variables from .env file
load_dotenv()

# Access the variables
openai_endpoint = os.environ.get('OPENAI_ENDPOINT')
openai_key = os.environ.get('OPENAI_KEY')
aisearch_endpoint = os.environ.get('AISEARCH_ENDPOINT')
aisearch_key = os.environ.get('AISEARCH_KEY')

openai_client = AzureOpenAI(
  api_key = openai_key,
  api_version = "2024-02-01",
  azure_endpoint = openai_endpoint
)

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai_client.embeddings.create(input = [text], model=model).data[0].embedding


In [80]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
import json
from openai import AzureOpenAI

openai_client = AzureOpenAI(
  api_key = openai_key,
  api_version = "2024-02-01",
  azure_endpoint = openai_endpoint
)

# Set up the connection
credential = AzureKeyCredential(aisearch_key)
client = SearchClient(endpoint=aisearch_endpoint, index_name='enterprise_docs', credential=credential)

documents = json.loads(open('documents.json').read())

for document in documents:
    document['embedding'] = get_embedding(document['content'])
    client.upload_documents(documents=[document])

print('Document uploaded successfully.')


Document uploaded successfully.


In [81]:
import json
# Define the query
def find_documents(text):
    query = {
        "vectorQueries": [
            {
            "kind": "text",
            "text": text,
            "k": 2,
            "fields": "embedding",
            }
        ]
    }

    # Perform the search
    response = client.search(query)

    # Convert the response to a list of dictionaries
    documents = [{'title': doc['title'], 'content': doc['content']} for doc in response]
    return str(documents)

In [85]:
import json
from pprint import pprint

def call_gpt(question):
  #question = "Do our Indian Pale Ale have Tomatoes?"
  documents = find_documents(question)

  # Print the documents variable in a pretty format
  print('-------------DOCUMENTS-------------')
  pprint(documents)
  print('-----------------')

  system_prompt = '''
  You are a helpful assistant working at a beer factory.
  YOU WILL ONLY ANSWER BASED ON THE 'DOCUMENTS' IN THE PROMPT.
  If the information is not in the documents, you answer with: Sorry, I dont know.
  Dont output the documents, only the answer.
  Respond very politely and friendly.
  DOCUMENTS: ''' + documents 

  response = openai_client.chat.completions.create(
    model="gpt-4",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": question}
      ]
  )

  #print(response.model_dump_json(indent=2))
  print('-------------RESPONSE-------------')
  print(response.choices[0].message.content)
  print('-----------------')

In [84]:
call_gpt("")

-------------DOCUMENTS-------------
'[]'
-----------------
-------------RESPONSE-------------
Sorry, I don't know.
-----------------
